# Test SSO Login

This notebook will test the authorization of the notebooks running in this library.

By including the `jupyterauth` module, the notebook will initialize the authorization of the notebooks. 
* For instance profile / machine configurations, the role details are displayed to the screen.
* For SSO configurations, the AWS SSO login will appear asking the user to grant access to the session.
* For role assumption, the default profile is set


In [1]:
import boto3, os
from jupyterirtools import jupyterauth


Logging in with IAM Identity Center....


Credentials expire in 7 hours and 5 minutes

The next cell will check all the configured permissions for this notebook. If the environment is configured for multiaccount via role assumption or sso, this will check each of the connections. 

In [2]:
jupyterauth.check_permissions()

### Account Status

|status|Account|Role|Status|
|-----|-----|-----|-----|
|<span style="color:green"> &#9679;</span>|959309154643|Jupyter-IR-AdministratorAccess|Successful|
|<span style="color:green"> &#9679;</span>|959309154643|Jupyter-IR-ViewOnly|Successful|
|<span style="color:green"> &#9679;</span>|913149361159|Jupyter-IR-AdministratorAccess|Successful|
|<span style="color:green"> &#9679;</span>|913149361159|Jupyter-IR-ViewOnly|Successful|
|<span style="color:green"> &#9679;</span>|251344881676|Jupyter-IR-AdministratorAccess|Successful|
|<span style="color:green"> &#9679;</span>|251344881676|Jupyter-IR-ViewOnly|Successful|
|<span style="color:green"> &#9679;</span>|383086473915|AdministratorAccess|Successful|
|<span style="color:green"> &#9679;</span>|383086473915|Jupyter-IR-AdministratorAccess|Successful|
|<span style="color:green"> &#9679;</span>|383086473915|Jupyter-IR-SysAdminAccess|Successful|
|<span style="color:green"> &#9679;</span>|383086473915|Jupyter-IR-ViewOnly|Successful|
|<span style="color:green"> &#9679;</span>|669399753706|Jupyter-IR-AdministratorAccess|Successful|
|<span style="color:green"> &#9679;</span>|669399753706|Jupyter-IR-ViewOnly|Successful|


This demonstrates how to use the `jupyterauth` module to iterate all the accounts and execute a boto command against every account. Here it is simply getting caller identity on each account.

In [3]:
for session, account in jupyterauth.get_session_by_account():
    sts = session.client('sts')
    identity = sts.get_caller_identity()
    print(f'Account: {account} User: {identity["UserId"]}')

Account: 959309154643 User: AROA56W2IBFJSYX4IEXFG:sso_user
Account: 959309154643 User: AROA56W2IBFJXJV7OFNRA:sso_user
Account: 913149361159 User: AROA5JG7RPQDUWWUJC2OU:sso_user
Account: 913149361159 User: AROA5JG7RPQD5R6GWMDDA:sso_user
Account: 251344881676 User: AROATVBKSRAGBDODPGT62:sso_user
Account: 251344881676 User: AROATVBKSRAGLFBLNEXHW:sso_user
Account: 383086473915 User: AROAVSMN3WK5YRE5BQOCL:sso_user
Account: 383086473915 User: AROAVSMN3WK57E362JKYS:sso_user
Account: 383086473915 User: AROAVSMN3WK5WENPTLT5U:sso_user
Account: 383086473915 User: AROAVSMN3WK5QHSKAELLD:sso_user
Account: 669399753706 User: AROAZXW2VUPVJOVGEKWRG:sso_user
Account: 669399753706 User: AROAZXW2VUPVB6S72X7EI:sso_user


The default session is configured based on the environment variables.

In [4]:
boto3.setup_default_session()
sts = boto3.client('sts')
identity = sts.get_caller_identity()
print(f"UserId: {identity['UserId']}\nAccount: {identity['Account']}\n")

UserId: AROA56W2IBFJSYX4IEXFG:sso_user
Account: 959309154643



In [5]:
session =jupyterauth.get_session()
sts = session.client('sts')
identity = sts.get_caller_identity()
print(f"UserId: {identity['UserId']}\nAccount: {identity['Account']}\n")

UserId: AROA56W2IBFJSYX4IEXFG:sso_user
Account: 959309154643



Works with CLI commands too.

In [6]:
! aws sts get-caller-identity


{
    "UserId": "AROA56W2IBFJSYX4IEXFG:sso_user",
    "Account": "959309154643",
    "Arn": "arn:aws:sts::959309154643:assumed-role/AWSReservedSSO_Jupyter-IR-AdministratorAccess_05d2fe07f4b1d8cd/sso_user"
}


In [ ]:
jupyterauth.logout()